In [8]:
!pip install --quiet icecream

In [9]:
import os
import sys

from icecream import ic
import glob
import matplotlib.pyplot as plt

import numpy as np
from scipy import ndimage
from skimage import color


In [10]:
if not os.path.isdir('/content/RAFT'):
  !git clone https://github.com/princeton-vl/RAFT.git
 
sys.path.insert(1,'/content/RAFT/core/')

from utils.frame_utils import readFlow as read_flow
from utils import flow_viz

In [23]:
if not os.path.isfile('/content/3vids_no_tensors.zip'):
    !wget https://www.dropbox.com/s/k2esmefb7sg8gxt/3vids_no_tensors.zip
    !unzip '/content/3vids_no_tensors.zip'

--2021-05-06 02:42:27--  https://www.dropbox.com/s/k2esmefb7sg8gxt/3vids_no_tensors.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/k2esmefb7sg8gxt/3vids_no_tensors.zip [following]
--2021-05-06 02:42:28--  https://www.dropbox.com/s/raw/k2esmefb7sg8gxt/3vids_no_tensors.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc33452e3f43235022ddaba229d.dl.dropboxusercontent.com/cd/0/inline/BN9fIw2d-lbckeydJMGo_fzopO9kZ1sk_GCW0DQxdFiJt8Zof4kNFrfRgaG15kyjTUUhON_u646N1fp2cMgcBVZdJ8G92099M3Ge35pikVUKQwRAy7gWJqoxUp_iDOy9bujogZJ5kfbKj_GHbn1ukYb8/file# [following]
--2021-05-06 02:42:28--  https://ucc33452e3f43235022ddaba229d.dl.dropboxusercontent.com/cd/0/inline/BN9fIw2d-lbckeydJMGo_fzopO9kZ1sk_GCW0DQxdFiJt8Zof4kNFrfRgaG15kyjTU

In [12]:
# Return a circular mask of (True/False) 2d array
def create_circular_mask(shape, x, y, r):
    h, w = shape

    Y, X = np.ogrid[:h, :w]
    dist_from_center_mat = np.sqrt((X - x)**2 + (Y - y)**2)

    mask = dist_from_center_mat <= r
    return mask

def get_masked_img(img, x, y, r):
  mask = create_circular_mask(img.shape, x, y, r)
  masked = img.copy()
  masked[~mask] = 0
  return masked

In [63]:
import numpy as np
from scipy import ndimage
from skimage import color

# from RAFT/core/utils
from utils.frame_utils import readFlow as read_flow
from utils import flow_viz

def get_xy_coord_for_flo(flo_im, show_plot=False):
    # flo_im = flow_viz.flow_to_image(read_flow(flo_file))
    if show_plot:
        ic("visualized flow file")
        plt.imshow(flo_im)
        plt.show()

    gray_flo = color.rgb2gray(flo_im)
    if show_plot:
        ic("grayscale flow")
        plt.imshow(gray_flo, cmap='gray')
        plt.show()

    # Invert the intensity so that "active" spots have higher numerical value in range [0, 1]
    gray_flo = (gray_flo - 1 ) * -1
    if show_plot:
        ic("grayscale flow (inverted high=1)")
        plt.imshow(gray_flo, cmap='gray')
        plt.show()
        ic(ndimage.measurements.center_of_mass(gray_flo))

    # Exponeniate the pixel values; they will still remain in [0, 1]
    gray_flo = gray_flo ** 4
    if show_plot:
        ic("exponentiated (** 4) grayscale flow")
        plt.imshow(gray_flo, cmap='gray')
        plt.show()
        ic(ndimage.measurements.center_of_mass(gray_flo))

    (h, w) = ndimage.measurements.center_of_mass(gray_flo)
    return (w, h)  # return in (x, y) format


======================================================================================================================

======================================================================================================================

# Detectron2 Beginner's Tutorial

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

Welcome to detectron2! This is the official colab tutorial of detectron2. Here, we will go through some basics usage of detectron2, including the following:
* Run inference on images or videos, with an existing detectron2 model
* Train a detectron2 model on a new dataset

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.


# Install detectron2

In [14]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 19.8MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=cae49009f46c68b2bae56f79fdf8aa43c7928dbebdd0abfd77b710c5d8246017
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.8.1+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [15]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.2MB 596kB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-cp37-none-any.whl size=58543 sha256=eff1c99d8b80bfa2ee4f397692fb3a8947d14289ca3dbd020ccbc44acd4ac161
  Stored in directory: /root/.cache/pip/wheels/d2/ee/3a/5c531df777c03d8c67f22c65f97d6f75321087482d05a9b218
Successfully built fvcore


In [16]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Run a pre-trained detectron2 model

In [18]:
# !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
# im = cv2.imread("./input.jpg")

# im = flo_0.copy()
# cv2_imshow(im)

Then, we create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [20]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
# outputs = predictor(im)

In [ ]:
# # look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
# ic(outputs["instances"].pred_classes.shape)
# ic(outputs["instances"].pred_classes)
# # https://detectron2.readthedocs.io/en/latest/modules/structures.html#detectron2.structures.Boxes
# ic(outputs["instances"].pred_boxes.tensor.shape)
# ic(outputs["instances"].pred_boxes)
# ic(outputs["instances"].pred_masks.shape)
# ic(outputs["instances"].pred_masks)
# print('')

In [ ]:
# # We can use `Visualizer` to draw the predictions on the image.
# v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
# out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# cv2_imshow(out.get_image()[:, :, ::-1])

# Using Detectron Bounding Box to Generate "radius" for given (x, y) coords

#Putting it all together

In [90]:
# One possible way to discern a radius is to take the average smallest
# dimension of each bounding box.
def avg_shortest_bbox_dimension(bbox_list):
    sum_of_shortest_dimensions = 0
    for x1, y1, x2, y2 in bbox_list:
        sum_of_shortest_dimensions += min(x2 - x1, y2 - y1)

    return sum_of_shortest_dimensions / len(bbox_list)


def generate_fove8n_labels(input_output_dirs, detector2_predictor):
    '''
    input_output_dirs: list of tuple of (input frame directory,
                                         input optical flow frame directory,
                                         output label file directory)
    detector2_predictor: a pre-trained, initialized detector2 predictor object
    '''
    for (frame_dir, flo_frame_dir, output_dir) in input_output_dirs:
        # Gather and sort all of the frame and optical flow files.
        frame_files = glob.glob(os.path.join(frame_dir, '*.png'))
        frame_files = sorted(frame_files)
        flo_files = glob.glob(os.path.join(flo_frame_dir, '*.flo'))
        flo_files = sorted(flo_files)
        assert len(frame_files) == len(flo_files) + 1, \
            "should be n and n - 1 frames: {} {}".format(frame_dir, flo_frame_dir)

        # Generate the candidate focal points for each optical flow frame.
        coords = []
        for flo_file in flo_files:
            flo_im = flow_viz.flow_to_image(read_flow(flo_file))
            (x, y) = get_xy_coord_for_flo(flo_im)  # returns (height, width)
            coords.append((x, y))

        # Pad the optic flow-derived coordinates with an extra duplicate,
        # representing the missing first frame.
        coords.insert(0, coords[0])

        # Run Detector2 to gather potential bounding boxes of items.
        bbox_lists = []
        for frame_file in frame_files:
            im = cv2.imread(frame_file)
            outputs = predictor(im)
            # https://detectron2.readthedocs.io/en/latest/modules/structures.html
            bbox_tensors =  outputs["instances"].pred_boxes.tensor
            bboxes = []
            for bbox_tensor in bbox_tensors:
                bboxes.append(bbox_tensor.tolist())
            bbox_lists.append(bboxes)

        assert len(coords) == len(bbox_lists)

        # Structure the bbox lists into bbox tuples for easier referencing.
        bbox_lists = [[(b[0], b[1], b[2], b[3]) for b in bbox_list] for bbox_list in bbox_lists]

        # Filter the bounding boxes for those that contain the focal point.
        filtered_bbox_lists = []
        for ((x, y), bboxes) in zip(coords, bbox_lists):
            containing_bboxes = []
            for (x1, y1, x2, y2) in bboxes:
                if x1 < x < x2 and y1 < y < y2:
                    containing_bboxes.append((x1, y1, x2, y2))
            filtered_bbox_lists.append(containing_bboxes)

        assert len(filtered_bbox_lists) == len(bbox_lists)

        # Generate guesses for reasonable radii.
        xyrs = []
        last_r = None
        for ((x, y), bboxes) in zip(coords, filtered_bbox_lists):
            # If there's no bounding box, just use the last radius.
            if len(bboxes) == 0:
                # If we haven't used a real radius yet, throw an error
                if not last_r:
                    raise Exception('No bounding boxes overlapped the focal point int the first frame: {}'.format(frame_dir))
                xyrs.append((x, y, last_r))
            else:
                current_r = avg_shortest_bbox_dimension(bboxes)
                xyrs.append((x, y, current_r))
                last_r = current_r

        # Write the labels as text files to the given directory.
        with open(os.path.join(output_dir, 'labels.txt'), 'w') as labels_file:
            for (x, y, r) in xyrs:
                label_line = "{} {} {}\n".format(x, y, r)
                labels_file.write(label_line)



In [91]:
tripping240_in_out = ('/content/3vids_no_tensors/frames/tripping240/',
                      '/content/3vids_no_tensors/opticFlow/tripping240/',
                      '/content/3vids_no_tensors/labels/tripping240/')

tripping339_in_out = ('/content/3vids_no_tensors/frames/tripping339/',
                      '/content/3vids_no_tensors/opticFlow/tripping339/',
                      '/content/3vids_no_tensors/labels/tripping339/')

tripping432_in_out = ('/content/3vids_no_tensors/frames/tripping432/',
                      '/content/3vids_no_tensors/opticFlow/tripping432/',
                      '/content/3vids_no_tensors/labels/tripping432/')
# problem with tripping240 -- no bounding box for first frame that overlaps focus area.
in_out_dirs = [tripping339_in_out, tripping432_in_out]

generate_fove8n_labels(in_out_dirs, predictor)